# 1. Preprocessing


In [22]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage.morphology import skeletonize
from skimage.util import img_as_ubyte
from scipy.ndimage import gaussian_filter
import math

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = gaussian_filter(gray, sigma=1.0)
    enhanced = cv2.equalizeHist(blurred)
    return enhanced


# 2. Custom Edge Detection (Gradient + Adaptive Threshold)

In [23]:
def custom_edge_detection(image):
    # Custom gradient filters
    kernel_x = np.array([[-1, 0, 1],
                         [-2, 0, 2],
                         [-1, 0, 1]])
    kernel_y = np.array([[-1, -2, -1],
                         [ 0,  0,  0],
                         [ 1,  2,  1]])

    grad_x = cv2.filter2D(image, -1, kernel_x)
    grad_y = cv2.filter2D(image, -1, kernel_y)

    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    magnitude = np.uint8(magnitude / magnitude.max() * 255)

    # Adaptive thresholding
    _, edge_map = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return edge_map

# 3. Leaf Segmentation

In [24]:
def segment_leaf(image):
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None, None
    largest_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(image)
    cv2.drawContours(mask, [largest_contour], -1, 255, -1)
    return mask, largest_contour

# 4. Feature Extraction

A. Shape Features

In [25]:
def extract_shape_features(contour):
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    area = cv2.contourArea(contour)
    rect_area = w * h
    extent = float(area) / rect_area
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area
    moments = cv2.moments(contour)
    hu_moments = cv2.HuMoments(moments).flatten()
    return [aspect_ratio, extent, solidity] + hu_moments.tolist()

B. Texture Features

In [26]:
def extract_texture_features(image, mask):
    masked = cv2.bitwise_and(image, image, mask=mask)
    lbp = local_binary_pattern(masked, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)

    glcm = graycomatrix(masked, [1], [0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    entropy = -np.sum(glcm * np.log2(glcm + 1e-10))

    return lbp_hist.tolist() + [contrast, homogeneity, entropy]


C. Vein Features

In [27]:
def extract_vein_features(mask):
    skeleton = skeletonize(mask // 255)
    vein_density = np.sum(skeleton) / np.sum(mask // 255)

    # Fractal dimension (box-counting method)
    def fractal_dimension(Z):
        def boxcount(Z, k):
            S = np.add.reduceat(
                np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                                   np.arange(0, Z.shape[1], k), axis=1)
            return len(np.where(S > 0)[0])
        Z = Z < 0.5
        p = min(Z.shape)
        n = 2**np.floor(np.log2(p))
        sizes = 2**np.arange(int(np.log2(n)), 1, -1)
        counts = [boxcount(Z, size) for size in sizes]
        coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
        return -coeffs[0]

    fractal_dim = fractal_dimension(skeleton)
    return [vein_density, fractal_dim]

D. Edge Features

In [28]:
def extract_edge_features(edge_map, mask):
    edge_masked = cv2.bitwise_and(edge_map, edge_map, mask=mask)
    edge_density = np.sum(edge_masked) / np.sum(mask)

    # Orientation histogram
    sobelx = cv2.Sobel(edge_masked, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(edge_masked, cv2.CV_64F, 0, 1, ksize=5)
    angles = np.arctan2(sobely, sobelx) * 180 / np.pi
    hist, _ = np.histogram(angles, bins=8, range=(-180, 180), density=True)

    return [edge_density] + hist.tolist()

# 5. Feature Fusion

In [29]:
def extract_all_features(image):
    preprocessed = preprocess_image(image)
    edge_map = custom_edge_detection(preprocessed)
    mask, contour = segment_leaf(preprocessed)

    if mask is None or contour is None:
        return None  # Not a leaf

    shape_feats = extract_shape_features(contour)
    texture_feats = extract_texture_features(preprocessed, mask)
    vein_feats = extract_vein_features(mask)
    edge_feats = extract_edge_features(edge_map, mask)

    feature_vector = shape_feats + texture_feats + vein_feats + edge_feats
    return feature_vector


In [36]:
image = cv2.imread('a.jpg')

features = extract_all_features(image)
print("Feature vector shape:", np.shape(features))
print("Sample features:", features[:5])  # Show first few values

print("Contains NaNs:", np.isnan(features).any())
print("Contains Infs:", np.isinf(features).any())


Feature vector shape: (34,)
Sample features: [1.8604651162790697, 0.9345220930232558, 0.963286269057436, 0.20412059912524852, 0.014292493816844741]
Contains NaNs: False
Contains Infs: False
